In [1]:
#《7自动求导》的代码
import torch
x = torch.arange(4.0,requires_grad=True)#requires_grad=True表示自动计算并存储梯度，之后可以使用.grad来访问梯度
print(x.grad)#默认值是None

None


In [2]:
y = 2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [3]:
#通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
#因为y = 2*x.T*x=2*[x1*x1+x2*x2+x3*x3+x4*x4],所以y关于各个x分量的梯度等于4*x
x.grad,x.grad == 4 * x

(tensor([ 0.,  4.,  8., 12.]), tensor([True, True, True, True]))

In [4]:
#默认情况下，pytorch会累加梯度，我们需要清除之前的值
x.grad.zero_()#清除梯度,注意是zero_()不是zero(),在后面加一个下划线表示用0替换原来向量中的元素
y = x.sum()
y.backward()
x.grad#因为y=x1+x2+x3+x4,所以y关于各个x分量的梯度都是1

tensor([1., 1., 1., 1.])

In [5]:
#深度学习中，我们的目的不是计算微分矩阵，而是批量中的每个样本单独计算的偏导数之和
#对于非标量函数需要转换成标量函数然后再进行反向传播求梯度，例如y=x*x
x.grad.zero_()
y = x * x#x*x是将x的各个元素对应相乘形成的向量即[x1^2,x2^2,x3^2,x4^2]
y.sum().backward()#使用sum()函数转换成标量函数
x.grad

tensor([0., 2., 4., 6.])

In [6]:
x.grad.zero_()
y = x * x
u = y.detach()#detach()表示将y退化成与x无关的量，也就是y不再是函数，而是一个值为[0., 1., 4., 9.]的向量
print(u)

tensor([0., 1., 4., 9.])


In [7]:
z = u * x#此时u就是一个常数数列，所以z对x求导就等于u
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [8]:
#即使构建函数的计算图需要通过python控制流（例如，条件，循环，或者任意函数调用），我们仍然可以计算得到变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000:#norm()函数用来求范数，对于向量默认求L2范数（求各元素平方和开方），对于矩阵，默认求Frobenius范数（求各元素平方和开方）
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100*b
    return c
a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()
a,a.grad,a.grad == d / a#此处a的梯度取决于a的值，a取不同的值，a前面的系数也就不同，不过最终a的梯度就是a前面的系数

(tensor(-0.7936, requires_grad=True), tensor(204800.), tensor(True))